In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import load_model
from keras import backend as K
import tensorflow as tf
from scipy.misc import imresize
import matplotlib.pyplot as plt
%matplotlib inline

# fix random seed for reproducibility
np.random.seed(7)


Using TensorFlow backend.


# Load data 

In [2]:
from keras.datasets import mnist
(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# split train data in two
X_pr = Xtrain[30000:60000, :, :]
Y_pr = Ytrain[30000:60000]

Xtrain = Xtrain[0:30000, :, :];
Ytrain = Ytrain[0:30000]

# downsample
factor = 1
if factor<1:
    Xtrain_down = np.ones((Xtrain.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(Xtrain.shape[0]):
        Xtrain_down[i, :, :] = imresize(Xtrain[i,:,:], factor)

    Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(Xtest.shape[0]):
        Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

    X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
    for i in range(X_pr.shape[0]):
        X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
else:
    Xtrain_down = Xtrain
    Xtest_down = Xtest
    X_pr_down = X_pr
    
# VECTORIZE IMAGES
Xtrain_down = Xtrain_down.reshape(Xtrain_down.shape[0], int(xdim*factor)**2).astype('float32') / 255
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255

# CATEGORICAL OUTPUTS
Ytrain_cat = np_utils.to_categorical(Ytrain, 10)
Ytest_cat = np_utils.to_categorical(Ytest, 10)


# TRAIN A FULLY-CONNECTED NN WITH 4 ReLU HIDDEN LAYERS 

In [3]:
# create model
model = Sequential()
model.add(Dense(128, input_shape=(int(xdim*factor)**2,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
model.fit(Xtrain_down, Ytrain_cat, nb_epoch=50, batch_size=16, verbose=2)

# evaluate the model
scores = model.evaluate(Xtest_down, Ytest_cat, verbose=0)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# Load NN and get output values
# from keras import backend as K
# from keras.models import load_model

# Save the model
#model.save('nonlinear_nn.h5')

# GET THE OUTPUT OF EACH LAYER AFTER TRAINING
inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = K.function([inp]+ [K.learning_phase()], outputs ) # evaluation function
layer_outs = functor([X_pr_down, 1.])                       # compute on 2nd training set
layer_outs_test = functor([Xtest_down, 1.])                 # compute on test set
#print(layer_outs)



/opt/anaconda/anaconda2/lib/python2.7/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/50
10s - loss: 0.2908 - acc: 0.9096
Epoch 2/50
8s - loss: 0.1326 - acc: 0.9601
Epoch 3/50
8s - loss: 0.0984 - acc: 0.9693
Epoch 4/50
8s - loss: 0.0760 - acc: 0.9767
Epoch 5/50
9s - loss: 0.0652 - acc: 0.9802
Epoch 6/50
8s - loss: 0.0557 - acc: 0.9833
Epoch 7/50
9s - loss: 0.0433 - acc: 0.9868
Epoch 8/50
8s - loss: 0.0453 - acc: 0.9857
Epoch 9/50
9s - loss: 0.0356 - acc: 0.9898
Epoch 10/50
8s - loss: 0.0322 - acc: 0.9902
Epoch 11/50
8s - loss: 0.0336 - acc: 0.9898
Epoch 12/50
8s - loss: 0.0291 - acc: 0.9911
Epoch 13/50
9s - loss: 0.0266 - acc: 0.9916
Epoch 14/50
8s - loss: 0.0241 - acc: 0.9933
Epoch 15/50
9s - loss: 0.0243 - acc: 0.9931
Epoch 16/50
9s - loss: 0.0189 - acc: 0.9942
Epoch 17/50
8s - loss: 0.0265 - acc: 0.9928
Epoch 18/50
8s - loss: 0.0177 - acc: 0.9951
Epoch 19/50
9s - loss: 0.0204 - acc: 0.9952
Epoch 20/50
10s - loss: 0.0225 - acc: 0.9942
Epoch 21/50
10s - loss: 0.0167 - acc: 0.9955
Epoch 22/50
10s - loss: 0.0218 - acc: 0.9938
Epoch 23/50
10s - loss: 0.0148 - acc:

# Save Layer Outputs

In [4]:
import pickle
with open('results/NN_layer_outputs.dat','wb') as f:
    pickle.dump([layer_outs, layer_outs_test], f)
f.close()